In [ ]:
import torch

def format_pytorch_version(version):
    return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
pip uninstall networkx

In [ ]:
pip install networkx==2.8.8

In [ ]:
pip install torchmetrics==0.9.3

---

In [ ]:
import time as tm
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch import nn
from torch.nn import Parameter
from torchmetrics import AUROC, F1Score
from torch_geometric.nn import GCNConv
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.utils.convert import from_networkx

In [ ]:
df      = pd.read_csv('y_train.csv')
y_train = torch.tensor(df['default']).type(torch.FloatTensor)

In [ ]:
G      = nx.read_gpickle("G_area.gpickle")
rs     = random.sample(list(G.nodes), int(0.5 * len(list(G.nodes))))
G.remove_edges_from(list(G.edges(rs)))
data   = from_networkx(G, group_node_attrs = all)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class GCN_LSTM(torch.nn.Module):

    def __init__(self, in_channels:int, out_channels:int, improved:bool = False, cached:bool = False,
                 add_self_loops:bool = True):        
        super(GCN_LSTM, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self._create_parameters_and_layers()
        self._set_parameters()
        
    def _create_input_gate_parameters_and_layers(self):
        self.conv_i = GCNConv(in_channels = self.out_channels, out_channels = self.out_channels, improved = self.improved,
                              cached = self.cached, add_self_loops = self.add_self_loops)
        self.W_i = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_i = Parameter(torch.Tensor(1, self.out_channels))

    def _create_forget_gate_parameters_and_layers(self):
        self.conv_f = GCNConv(in_channels = self.out_channels, out_channels = self.out_channels, improved = self.improved,
                              cached = self.cached, add_self_loops = self.add_self_loops)
        self.W_f = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_f = Parameter(torch.Tensor(1, self.out_channels))

    def _create_cell_state_parameters_and_layers(self):
        self.conv_c = GCNConv(in_channels = self.out_channels, out_channels = self.out_channels, improved = self.improved,
                              cached = self.cached, add_self_loops = self.add_self_loops)
        self.W_c = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_c = Parameter(torch.Tensor(1, self.out_channels))

    def _create_output_gate_parameters_and_layers(self):
        self.conv_o = GCNConv(in_channels = self.out_channels, out_channels = self.out_channels, improved = self.improved,
                              cached = self.cached, add_self_loops = self.add_self_loops)
        self.W_o = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_o = Parameter(torch.Tensor(1, self.out_channels))

    def _create_parameters_and_layers(self):
        self._create_input_gate_parameters_and_layers()
        self._create_forget_gate_parameters_and_layers()
        self._create_cell_state_parameters_and_layers()
        self._create_output_gate_parameters_and_layers()

    def _set_parameters(self):
        glorot(self.W_i)
        glorot(self.W_f)
        glorot(self.W_c)
        glorot(self.W_o)
        zeros(self.b_i)
        zeros(self.b_f)
        zeros(self.b_c)
        zeros(self.b_o)

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _set_cell_state(self, X, C):
        if C is None:
            C = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return C

    def _calculate_input_gate(self, X, edge_index, H, C):
        I = torch.matmul(X, self.W_i)
        I = I + self.conv_i(H, edge_index)
        I = I + self.b_i
        I = torch.sigmoid(I)
        return I

    def _calculate_forget_gate(self, X, edge_index, H, C):
        F = torch.matmul(X, self.W_f)
        F = F + self.conv_f(H, edge_index)
        F = F + self.b_f
        F = torch.sigmoid(F)
        return F

    def _calculate_cell_state(self, X, edge_index, H, C, I, F):
        T = torch.matmul(X, self.W_c)
        T = T + self.conv_c(H, edge_index)
        T = T + self.b_c
        T = torch.tanh(T)
        C = F * C + I * T
        return C

    def _calculate_output_gate(self, X, edge_index, H, C):
        O = torch.matmul(X, self.W_o)
        O = O + self.conv_o(H, edge_index)
        O = O + self.b_o
        O = torch.sigmoid(O)
        return O

    def _calculate_hidden_state(self, O, C):
        H = O * torch.tanh(C)
        return H

    def forward(self, X:torch.FloatTensor, edge_index:torch.LongTensor, H:torch.FloatTensor = None,
                C:torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X, H)
        C = self._set_cell_state(X, C)
        I = self._calculate_input_gate(X, edge_index, H, C)
        F = self._calculate_forget_gate(X, edge_index, H, C)
        C = self._calculate_cell_state(X, edge_index, H, C, I, F)
        O = self._calculate_output_gate(X, edge_index, H, C)
        H = self._calculate_hidden_state(O, C)
        return H, C

In [ ]:
class GCN_LSTM_ATT(torch.nn.Module):

    def __init__(self, in_channels:int, out_channels:int, periods:int, improved:bool = False, cached:bool = False,
                 add_self_loops:bool = True):
        super(GCN_LSTM_ATT, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.periods = periods
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self._setup_layers()

    def _setup_layers(self):
        self._base_gcn_lstm = GCN_LSTM(in_channels = self.in_channels, out_channels = self.out_channels,
                                       improved = self.improved, cached = self.cached, add_self_loops = self.add_self_loops)
        self._attention = torch.nn.Parameter(torch.empty(self.periods, device = device))
        torch.nn.init.uniform_(self._attention)

    def forward(self, X1:torch.FloatTensor, edge_index1:torch.LongTensor,
                      X2:torch.FloatTensor, edge_index2:torch.LongTensor,
                      X3:torch.FloatTensor, edge_index3:torch.LongTensor,
                      X4:torch.FloatTensor, edge_index4:torch.LongTensor,
                      X5:torch.FloatTensor, edge_index5:torch.LongTensor,
                      X6:torch.FloatTensor, edge_index6:torch.LongTensor,
                      H:torch.FloatTensor = None, C:torch.FloatTensor = None) -> torch.FloatTensor:
        H_accum = 0
        probs = torch.nn.functional.softmax(self._attention, dim = 0)
        for period in range(self.periods):
            if period == 0:
                X = X1
                edge_index = edge_index1
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            elif period == 1:
                X = X2
                edge_index = edge_index2
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            elif period == 2:
                X = X3
                edge_index = edge_index3
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            elif period == 3:
                X = X4
                edge_index = edge_index4
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            elif period == 4:
                X = X5
                edge_index = edge_index5
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            elif period == 5:
                X = X6
                edge_index = edge_index6
                H, C = self._base_gcn_lstm(X, edge_index, H, C)
            H_accum = H_accum + probs[period] * H
        return H_accum

In [ ]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GCN_LSTM_ATT(node_features, --, periods)
        self.linear_1 = torch.nn.Linear(--, --)
        self.dropout = torch.nn.Dropout(p = --)
        self.linear_2 = torch.nn.Linear(--, --)

    def forward(self, x1, edge_index1,
                      x2, edge_index2,
                      x3, edge_index3,
                      x4, edge_index4,
                      x5, edge_index5,
                      x6, edge_index6):
        h = self.recurrent(x1, edge_index1,
                           x2, edge_index2,
                           x3, edge_index3,
                           x4, edge_index4,
                           x5, edge_index5,
                           x6, edge_index6)
        h = self.linear_1(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.linear_2(h)
        return torch.sigmoid(h)

model = RecurrentGCN(node_features = 16, periods = 6).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
st = tm.time()

model.train()
for epoch in tqdm(range(--)):
    for w in range(13):
        for s in range(6):
            exec("x" + str(s + 1) + " = " + "data" + str(w + s + 1) + ".x.to(device)")
            exec("edge_index" + str(s + 1) + " = " + "data" + str(w + s + 1) + ".edge_index.to(device)")
        y_targ = y_train.to(device)
        y_pred = model(x1, edge_index1,
                       x2, edge_index2,
                       x3, edge_index3,
                       x4, edge_index4,
                       x5, edge_index5,
                       x6, edge_index6)
        cost = criterion(y_pred, y_targ.reshape(-1, 1))
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
    if epoch % 100 == 0:
        print('cost = ', cost.item())
        print('accuracy = ', (torch.round(y_pred) == y_targ.reshape(-1, 1)).sum().item() / len(y_targ))

et = tm.time()
elapsed_time = et - st
print('\ntraining time:', elapsed_time, 'seconds')

In [ ]:
df_test = pd.read_csv('y_test.csv')
y_test  = torch.tensor(df_test['default']).type(torch.FloatTensor)

In [ ]:
G_test      = nx.read_gpickle("G_area_test.gpickle")
rs          = random.sample(list(G_test.nodes), int(0.5 * len(list(G_test.nodes))))
G_test.remove_edges_from(list(G_test.edges(rs)))
data_test   = from_networkx(G_test, group_node_attrs = all)

In [ ]:
for s in range(6):
    exec("x" + str(s + 1) + "_test = " + "data" + str(s + 19) + "_test.x.to(device)")
    exec("edge_index" + str(s + 1) + "_test = " + "data" + str(s + 19) + "_test.edge_index.to(device)")

y_targ_test = y_test

In [ ]:
model.eval()
auroc = AUROC(pos_label = 1)
f1score = F1Score(num_classes = 2, multiclass = True)
with torch.no_grad():
    preds = model(x1_test, edge_index1_test,
                  x2_test, edge_index2_test,
                  x3_test, edge_index3_test,
                  x4_test, edge_index4_test,
                  x5_test, edge_index5_test,
                  x6_test, edge_index6_test).cpu()
    target = y_targ_test.reshape(-1, 1).type(torch.int64)
    auroc_test = auroc(preds, target).item()
    f1score_test = f1score(preds, target).item()
    error_test = criterion(preds, y_targ_test.reshape(-1, 1)).item()

In [ ]:
print('AUC = ', auroc_test)
print('F1-score = ', f1score_test)
print('Error = ', error_test)